<a href="https://colab.research.google.com/github/DamiamAlfaro/Accuracy/blob/main/DIRSearchResultsExcelAllocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openpyxl
from google.colab import files

upload = files.upload()

Saving FourthTry.txt to FourthTry (2).txt


In [ ]:
search_results_raw_text = "FourthTry (2).txt"

data = []

with open(search_results_raw_text,"r") as file:
  read = file.read()
  projects_raw = read.split("Project Number:")

projects_text = projects_raw[1:]

projects = []
for project_raw in projects_text:
  projects.append(project_raw.split("\n"))

# Removes unnessary items at the end of
# the lists, such as "".
for cutting in range(len(projects)):
  while projects[cutting][-1] == "":
    projects[cutting].pop()


In [ ]:
# Demonstration
lengths = []
for demo1 in projects:
  print(demo1)
  lengths.append(len(demo1))
  for demo1_2 in demo1:
    print(f'{demo1_2}\n======================================')
  print("++++++++++++++++++++++++++++++++++++++++++++++")

print(lengths)
print(projects[1])
print(projects[3])

Streaming output truncated to the last 5000 lines.
Salinas, CA 93906	Advertised: 03/21/2019
Award: 03/21/2019
Est. Start: 03/21/2019
Est. Comp: 05/03/2019	LABORERS	MONTEREY

Contractor	
Sub Contractor
PWCR/CSLB/Lic	Name	PWCR/CSLB/Lic	Name
1000004387	STAPLES CONSTRUCTION COMPANY INC.	1000012273	SAN JOSE BOILER WORKS, INC
++++++++++++++++++++++++++++++++++++++++++++++
['', 'JF02-16-25-J8595-00', 'DIR Project', 'ID: 210978\tCounty of Monterey RMA - Public Works\t1412 Natividad - Adult Rehabilitation - Replace 2 Camus Boilers', '', 'Amount:', '$134,772.15\t1422 Natividad', 'Salinas , CA 93905\tAdvertised: 08/07/2017', 'Award: 08/30/2017', 'Est. Start: 08/30/2017', 'Est. Comp: 10/31/2017\tBOILERMAKER', 'LABORERS', 'SHEET METAL\tMONTEREY', '', 'Contractor\t', 'Sub Contractor', 'PWCR/CSLB/Lic\tName\tPWCR/CSLB/Lic\tName', '1000004387\tSTAPLES CONSTRUCTION COMPANY INC.\t1000012273\tSAN JOSE BOILER WORKS, INC']

JF02-16-25-J8595-00
DIR Project
ID: 210978	County of Monterey RMA - Public Works	141

In [ ]:
count = 0
for project in range(len(projects)):
  count += 1
  """
  Data simplified and ready to be moved to excel
  """
  projectAll = []
  """
  The dicitonary containing individual project information
  that will eventually go to the excel file.
  Please refer above for the information type.
  """
  projectInfo = {}

  # Project Number
  projectInfo["ProjectNumber"] = projects[project][1]
  """
  Breaks item [3] into three:
  1) DIR ID Number
  2) Awarding Body
  3) Name of the Project
  """
  exosphere = projects[project][3].split("\t")

  # Project DIR Number
  projectInfo["DIRNumber"] = exosphere[0]

  # Project Awarding Body
  projectInfo["AwardingBody"] = exosphere[1]

  # Name of Project
  projectInfo["ProjectName"] = exosphere[2]
  """
  Breaks item [6],[7]->[7.1],[8],[9],[10]->[10.1]
  [6]:
  1) Amount <--
  2) Address Street <--+
  [7]:
  1) Address Remanining <--+
  2) Advertised Date [7.1]
  [7.1]:
  1) 'Advertised'
  2) Date Advertised <--
  [8]:
  1) 'Award'
  2) Date Awarded <--
  [9]:
  1) 'Start'
  2) Date Started <--
  [10]:
  1) Completition Date [10.1]
  2) Classification <--
  3) County <--
  [10.1]
  1) 'Est. Comp'
  2) Completition Date <--
  """
  # [6]
  thermosphere = projects[project][6].split("\t")

  projectInfo["Amount"] = thermosphere[0]

  address = ""
  address += f'{thermosphere[1]} '

  # [7]
  mesosphere = projects[project][7].split("\t")
  # Scenario A: Extra Address Attributes (POs, Apartments, etc)
  if len(mesosphere) < 2:
    # [7.1]
    address += f'{mesosphere[0]} '
    mesosphere2 = projects[project][8].split("\t")
    address += f'{mesosphere2[0]}'
    projectInfo["Address"] = address
    stratosphere2 = mesosphere2[1].split(":")
    projectInfo["DateAdvertised"] = stratosphere2[1]
    # [8]
    troposphere2 = projects[project][9].split(":")
    projectInfo["AwardedDate"] = troposphere2[1]
    # [9]
    continentalCrust2 = projects[project][10].split(":")
    projectInfo["StartDate"] = continentalCrust2[1]
    # [10]
    oceanincCrust2 = projects[project][11].split("\t")
    # [10.1]
    mantle2 = oceanincCrust2[0].split(":")
    projectInfo["CompletitionDate"] = mantle2[1]

    # ------------
    # Scenario A: Equaling to three means only one classification
    if len(oceanincCrust) == 3:
      projectInfo["Classifications"] = oceanincCrust[1]
      projectInfo["County"] = oceanincCrust[2]

      # Allocating Contractor & Subcontractor intel
      beginning1 = 16
      entitiesInvolved1 = projects[project][beginning1].split("\t")

      # One subcontractor or more
      if len(entitiesInvolved1) > 2:
        subcontractorCount1 = 1
        projectInfo["GCIDNumber"] = entitiesInvolved1[0]
        projectInfo["GCName"] = entitiesInvolved1[1]
        projectInfo[f"Subcontractor{subcontractorCount1}IDNumber"] = entitiesInvolved1[2]
        projectInfo[f"Subcontractor{subcontractorCount1}Name"] = entitiesInvolved1[3]
        subcontractorCount1 += 1

        # Adding the rest of the subcontractors
        x2 = beginning1 + 1
        y2 = len(projects[project])
        z2 = x2

        for remainingSubs1 in range(x2,y2):

          # Last one on the list
          if z2 - y2 == -1:
            lastOne = projects[project][remainingSubs1].split("\t")
            projectInfo[f"Subcontractor{subcontractorCount1}IDNumber"] = lastOne[0]
            projectInfo[f"Subcontractor{subcontractorCount1}Name"] = lastOne[1]

          # Remaining ones
          else:
            splitting = projects[project][remainingSubs1].split("\t")
            projectInfo[f"Subcontractor{subcontractorCount1}IDNumber"] = splitting[0]
            projectInfo[f"Subcontractor{subcontractorCount1}Name"] = splitting[1]
            z2 += 1
            subcontractorCount1 += 1

      # Just the GC; no subcontractors
      else:
        projectInfo["GCIDNumber"] = entitiesInvolved1[0]
        projectInfo["GCName"] = entitiesInvolved1[1]


    # Scenario B: More than one classification
    elif len(oceanincCrust) == 2:
      classifications = []
      classifications.append(oceanincCrust[1])
      """
      We do not know how extensive the list of classifications will be.
      Therefore we are starting from the second one all the way to
      when the County appears. Why the second one? Because the first one
      is found in the OceanicCrust.
      """
      # -9 because that is where the next classification starts
      x = (len(projects[project])) - 9
      y = len(projects[project])
      # Where new outset begins
      index = 0
      county = ""
      for classification in range(x,y):
        if projects[project][classification] == "":
          index += classification
          break
        elif '\t' in projects[project][classification]:
          stratum1 = projects[project][classification].split("\t")
          classifications.append(stratum1[0])
          county += stratum1[1]
        else:
          classifications.append(projects[project][classification])

      projectInfo["Classifications"] = classifications
      projectInfo["County"] = county

      # Allocating Contractor & Subcontractor intel
      # index = 15 now
      entitiesInvolved2 = projects[project][index+4].split("\t")

      # One Subcontractor or more
      if len(entitiesInvolved2) > 2:
        subcontractorCount2 = 1
        projectInfo["GCIDNumber"] = entitiesInvolved2[0]
        projectInfo["GCName"] = entitiesInvolved2[1]
        projectInfo[f"Subcontractor{subcontractorCount2}IDNumber"] = entitiesInvolved2[2]
        projectInfo[f"Subcontractor{subcontractorCount2}Name"] = entitiesInvolved2[3]
        subcontractorCount2 += 1

        # Adding the rest of the subcontractors
        x3 = index+3
        y3 = len(projects[project])
        z3 = x3


        for remainingSubs2 in range(x3,y3):

          # Last one on the list
          if z3 - y3 == -1:
            lastOne = projects[project][remainingSubs2].split("\t")
            projectInfo[f"Subcontractor{subcontractorCount2}IDNumber"] = lastOne[0]
            projectInfo[f"Subcontractor{subcontractorCount2}Name"] = lastOne[1]

          # Remaining ones
          else:
            splitting = projects[project][remainingSubs2].split("\t")
            projectInfo[f"Subcontractor{subcontractorCount2}IDNumber"] = splitting[0]
            projectInfo[f"Subcontractor{subcontractorCount2}Name"] = splitting[1]
            z3 += 1
            subcontractorCount2 += 1

      # Just the GC; no subcontractors
      else:
        projectInfo["GCIDNumber"] = entitiesInvolved2[0]
        projectInfo["GCName"] = entitiesInvolved2[1]

    else:
      raise "\nSomething's up here...\n"

  # Scenario B: No Extra Attributes
  else:
  # [7.1]
    address += mesosphere[0]
    projectInfo["Address"] = address
    stratosphere = mesosphere[1].split(":")
    projectInfo["DateAdvertised"] = stratosphere[1]
    # [8]
    troposphere = projects[project][8].split(":")
    projectInfo["AwardedDate"] = troposphere[1]
    # [9]
    continentalCrust = projects[project][9].split(":")
    projectInfo["StartDate"] = continentalCrust[1]
    # [10]
    oceanincCrust = projects[project][10].split("\t")

    # [10.1]
    mantle = oceanincCrust[0].split(":")
    projectInfo["CompletitionDate"] = mantle[1]

    # Scenario A: Equaling to three means only one classification
    if len(oceanincCrust) == 3:
      projectInfo["Classifications"] = oceanincCrust[1]
      projectInfo["County"] = oceanincCrust[2]

      # Allocating Contractor & Subcontractor intel
      outset1 = 15
      entitiesInvolved1 = projects[project][outset1].split("\t")

      # One subcontractor or more
      if len(entitiesInvolved1) > 2:
        subcontractorCount1 = 1
        projectInfo["GCIDNumber"] = entitiesInvolved1[0]
        projectInfo["GCName"] = entitiesInvolved1[1]
        projectInfo[f"Subcontractor{subcontractorCount1}IDNumber"] = entitiesInvolved1[2]
        projectInfo[f"Subcontractor{subcontractorCount1}Name"] = entitiesInvolved1[3]
        subcontractorCount1 += 1

        # Adding the rest of the subcontractors
        x2 = outset1 + 1
        y2 = len(projects[project])
        z2 = x2

        for remainingSubs1 in range(x2,y2):

          # Last one on the list
          if z2 - y2 == -1:
            lastOne = projects[project][remainingSubs1].split("\t")
            projectInfo[f"Subcontractor{subcontractorCount1}IDNumber"] = lastOne[0]
            projectInfo[f"Subcontractor{subcontractorCount1}Name"] = lastOne[1]

          # Remaining ones
          else:
            splitting = projects[project][remainingSubs1].split("\t")
            projectInfo[f"Subcontractor{subcontractorCount1}IDNumber"] = splitting[0]
            projectInfo[f"Subcontractor{subcontractorCount1}Name"] = splitting[1]
            z2 += 1
            subcontractorCount1 += 1

      # Just the GC; no subcontractors
      else:
        projectInfo["GCIDNumber"] = entitiesInvolved1[0]
        projectInfo["GCName"] = entitiesInvolved1[1]


    # Scenario B: More than one classification
    elif len(oceanincCrust) == 2:
      classifications = []
      classifications.append(oceanincCrust[1])
      """
      We do not know how extensive the list of classifications will be.
      Therefore we are starting from the second one all the way to
      when the County appears. Why the second one? Because the first one
      is found in the OceanicCrust.
      """
      # -9 because that is the index where next classification starts
      x = (len(projects[project])) - 9
      y = len(projects[project])
      # Where new outset begins
      index = 0
      county = ""
      for classification in range(x,y):
        if projects[project][classification] == "":
          index += classification
          break
        elif '\t' in projects[project][classification]:
          stratum1 = projects[project][classification].split("\t")
          classifications.append(stratum1[0])
          county += stratum1[1]
        else:
          classifications.append(projects[project][classification])

      projectInfo["Classifications"] = classifications
      projectInfo["County"] = county

      # Allocating Contractor & Subcontractor intel

      # Continuation is where the GC's name appears
      continuation = index+4
      entitiesInvolved2 = projects[project][continuation].split("\t")




      # One Subcontractor or more
      check = continuation - len(projects[project])
      if len(entitiesInvolved2) == 4 and check < -1:
        subcontractorCount2 = 1
        projectInfo["GCIDNumber"] = entitiesInvolved2[0]
        projectInfo["GCName"] = entitiesInvolved2[1]
        projectInfo[f"Subcontractor{subcontractorCount2}IDNumber"] = entitiesInvolved2[2]
        projectInfo[f"Subcontractor{subcontractorCount2}Name"] = entitiesInvolved2[3]
        subcontractorCount2 += 1


        # Adding the rest of the subcontractors
        x3 = continuation+1
        y3 = len(projects[project])
        z3 = x3

        for remainingSubs2 in range(x3,y3):


          # Last one on the list
          if z3 - y3 == -1:
            lastOne = projects[project][remainingSubs2].split("\t")
            projectInfo[f"Subcontractor{subcontractorCount2}IDNumber"] = lastOne[0]
            projectInfo[f"Subcontractor{subcontractorCount2}Name"] = lastOne[1]
          # Remaining ones
          else:
            splitting = projects[project][remainingSubs2].split("\t")
            projectInfo[f"Subcontractor{subcontractorCount2}IDNumber"] = splitting[0]
            projectInfo[f"Subcontractor{subcontractorCount2}Name"] = splitting[1]
            z3 += 1
            subcontractorCount2 += 1

      # Just the GC and solely one subcontractor
      elif len(entitiesInvolved2) == 4 and check == -1:
        subcontractorCount2 = 1
        projectInfo["GCIDNumber"] = entitiesInvolved2[0]
        projectInfo["GCName"] = entitiesInvolved2[1]
        projectInfo[f"Subcontractor{subcontractorCount2}IDNumber"] = entitiesInvolved2[2]
        projectInfo[f"Subcontractor{subcontractorCount2}Name"] = entitiesInvolved2[3]

      # Just the GC; no subcontractors
      else:
        projectInfo["GCIDNumber"] = entitiesInvolved2[0]
        projectInfo["GCName"] = entitiesInvolved2[1]

    else:
      raise "\nSomething's up here...\n"






  projectAll.append(projectInfo)
  print(projectAll)
  #data.append(projectAll)


[{'ProjectNumber': 'JB02-23-04', 'DIRNumber': 'ID: 508427', 'AwardingBody': 'County of Santa Cruz', 'ProjectName': '1060 Emeline Window Replacement', 'Amount': '$138,969.83', 'Address': '1060 Emeline Ave Santa Cruz, CA 95060', 'DateAdvertised': ' 09/12/2023', 'AwardedDate': ' 09/12/2023', 'StartDate': ' 09/25/2023', 'CompletitionDate': ' 12/29/2023', 'Classifications': ['CARPENTERS', 'Est. Comp: 12/29/2023', 'LABORERS'], 'County': 'CARPENTERSSANTA CRUZ', 'GCIDNumber': '1000004387', 'GCName': 'STAPLES CONSTRUCTION COMPANY INC.', 'Subcontractor1IDNumber': '1000421264', 'Subcontractor1Name': 'COAST COUNTIES GLASS', 'Subcontractor2IDNumber': '1000010542', 'Subcontractor2Name': 'WM B SALEH PAINTING INC', 'Subcontractor3IDNumber': '1000007539', 'Subcontractor3Name': 'PENINSULATORS, INC.'}]
[{'ProjectNumber': '8595', 'DIRNumber': 'ID: 309889', 'AwardingBody': 'County of Monterey RMA - Public Works', 'ProjectName': '1270 Natividad - HVAC & BIO Safety System', 'Amount': '$171,662.00', 'Address'

IndexError: list index out of range

In [ ]:
print(len(data))
print(data)

z = 1
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = 'Dictionary Data'


start_row = 1

for list_item in data:
    dict_item = list_item[0]  # Access the dictionary

    # Write keys in column A and values in column B
    for key, value in dict_item.items():
        # Convert list to comma-separated string if value is a list
        if isinstance(value, list):
            value = ', '.join(value)  # Join list elements with a comma

        # Write the key
        sheet.cell(row=start_row, column=1, value=key)

        # Write the value
        sheet.cell(row=start_row, column=2, value=value)

        # Move to the next row
        start_row += 1

# Save the workbook
workbook.save("IamTheBest1.xlsx")

#files.download("IamTheBest1.xlsx")


10
[[{'ProjectNumber': 'JF02-16-14-J8184', 'DIRNumber': 'ID: 207242', 'AwardingBody': 'County of Monterey RMA - Public Works', 'ProjectName': '315 El Camino Real - Greenfield Library ADA Sink Imprv', 'Amount': '$2,946.21', 'Address': '315 El Camino Real Greenfield, CA 93927', 'DateAdvertised': ' 08/01/2017', 'AwardedDate': ' 08/02/2017', 'StartDate': ' 09/04/2017', 'CompletitionDate': ' 09/08/2017', 'Classifications': 'LABORERS', 'County': 'MONTEREY', 'GCIDNumber': '1000004387', 'GCName': 'STAPLES CONSTRUCTION COMPANY INC.'}], [{'ProjectNumber': '2000000450', 'DIRNumber': 'ID: 397336', 'AwardingBody': 'CSU Sacramento LD', 'ProjectName': 'J30-21-0002 Replaced Library Table', 'Amount': '$4,718.34', 'Address': '6000 J Street MS 6008 Sacramento, CA 95819', 'DateAdvertised': ' 03/08/2021', 'AwardedDate': ' 12/01/2021', 'StartDate': ' 12/01/2021', 'CompletitionDate': ' 03/01/2022', 'Classifications': 'LABORERS', 'County': 'MONTEREY', 'GCIDNumber': '1000004387', 'GCName': 'STAPLES CONSTRUCTIO